5. Работа с векторами слов

Наборы вещественных чисел, отражающие смысл слов естественного языка, называются векторами слов:

В этой главе векторы слов будут использоваться для вычисления семантического подобия различных текстов, что позволит категоризировать тексты в соответствии с их тематикой.

Пространство векторов слов можно представить в виде облака, где векторы располагаются тем ближе, чем ближе значения

#### Метод similarity

similarity, который используется для вычисления оценки семантического подобия двух объектов-контейнеров произвольного типа посредством сравнения их векторов слов.

In [1]:
import spacy as sp
nlp = sp.load('ru_core_news_lg')

In [7]:
doc = nlp("Я хочу на море")
clice = doc[2:5] # на море

In [8]:
doc.similarity(clice) # схожесть от 0 до 1. 0,88 очень высоко

0.9103373697881717

In [9]:
doc2=nlp('Я хочу на мандарин')

In [10]:
doc.similarity(doc2)

0.3896183839812772